In [1]:
import pandas as pd

In [2]:
clubs = pd.read_csv('clubs.csv')
clubs.head()

,club_id,club_code
0,127,sc-paderborn-07
1,192,roda-jc-kerkrade
2,2239,us-cremonese
3,2423,cd-santa-clara
4,24245,umraniyespor


In [3]:
comp = pd.read_csv('competitions.csv')
comp

,competition_id,competition_code,name,sub_type,type,country_name,domestic_league_code
0,BE1,jupiler-pro-league,jupiler-pro-league,first_tier,domestic_league,Belgium,BE1
1,SCI,supercoppa-italiana,supercoppa-italiana,domestic_super_cup,other,Italy,IT1
2,SFA,sfa-cup,sfa-cup,domestic_cup,domestic_cup,Scotland,SC1
3,UKRS,ukrainian-super-cup,ukrainian-super-cup,domestic_super_cup,other,Ukraine,UKR1
4,FAC,fa-cup,fa-cup,domestic_cup,domestic_cup,England,GB1
5,KLUB,fifa-klub-wm,fifa-klub-wm,fifa_club_world_cup,other,NaN,NaN
6,BESC,belgian-supercup,belgian-supercup,domestic_super_cup,other,Belgium,BE1
7,CGB,efl-cup,efl-cup,league_cup,other,England,GB1
8,DK1,superligaen,superligaen,first_tier,domestic_league,Denmark,DK1
9,ECLQ,uefa-europa-conference-league-qualifikation,uefa-europa-conference-league-qualifikation,uefa_europa_conference_league_qualifiers,international_cup,NaN,NaN


In [4]:
pv = pd.read_csv('player_valuations.csv').drop(['dateweek','datetime','current_club_id',
                                            'n'], axis=1)
pv.head()

,player_id,last_season,date,market_value_in_eur,player_club_domestic_competition_id
0,3132,2013,12/9/03,400000,TR1
1,6893,2012,12/15/03,900000,GB1
2,10,2015,10/4/04,7000000,IT1
3,26,2017,10/4/04,1500000,L1
4,65,2015,10/4/04,8000000,GR1


In [5]:
p = pd.read_csv('players.csv').drop(['market_value_in_eur','last_season','city_of_birth','url',
                                     'country_of_citizenship','agent_name','image_url',
                                     'highest_market_value_in_eur','position'], axis=1)
p = pd.merge(p,clubs,left_on='current_club_id', right_on='club_id')
new = pd.merge(p, pv, on=['player_id'])
new['date'] = pd.to_datetime(new['date'])
new = new.sort_values(['player_id', 'date']).drop_duplicates('player_id', keep='last')
new = new[new['last_season']==2022]
          
# Filter out rows with specified values in 'current_club_domestic_competition_id'
new = new[~new['current_club_domestic_competition_id'].isin(['TR1', 'GR1', 'DK1', 'UKR1'])]

#special cases
new['player_code'] = new['player_code'].replace({'nacho-fernandez':'nacho'})
new['Born']=new['date_of_birth'].str[-2:].astype(str)
new['player_code'] = new['player_code'].str.replace('ƒá','c')
new['player_code'] = new['player_code'] + new['Born']
new['Based'] = new['current_club_domestic_competition_id'].replace({'IT1': 'Serie A', 'GB1': 'Premier League', 
                                                                   'PO1':'Liga Nos','BE1':'Jupiler Pro League',
                                                                   'L1':'Bundesliga','GR1':'Greece','TR1':'Super Lig',
                                                                   'ES1':'La Liga','DK1':'Denmark','NL1':'Eredivisie',
                                                                   'RU1':'Premier Liga','SC1':'Scottish Premiership',
                                                                   'FR1':'Ligue 1','UKR1':'Ukraine'})
new

,player_id,first_name,last_name,name,current_club_id,player_code,country_of_birth,date_of_birth,sub_position,foot,...,current_club_domestic_competition_id,current_club_name,club_id,club_code,last_season,date,market_value_in_eur,player_club_domestic_competition_id,Born,Based
359754,3333,James,Milner,James Milner,31,james-milner86,England,1/4/86,Central Midfield,right,...,GB1,Liverpool FC,31,fc-liverpool,2022,2022-11-03,2000000,GB1,86,Premier League
149285,3455,Zlatan,Ibrahimoviƒá,Zlatan Ibrahimoviƒá,5,zlatan-ibrahimovic81,Sweden,10/3/81,Centre-Forward,right,...,IT1,AC Milan,5,ac-mailand,2022,2022-11-11,2000000,IT1,81,Serie A
5413,4311,Maarten,Stekelenburg,Maarten Stekelenburg,610,maarten-stekelenburg82,Netherlands,9/22/82,Goalkeeper,right,...,NL1,Ajax Amsterdam,610,ajax-amsterdam,2022,2022-11-11,100000,NL1,82,Eredivisie
414713,4391,Boy,Waterman,Boy Waterman,383,boy-waterman84,Netherlands,1/24/84,Goalkeeper,right,...,NL1,PSV Eindhoven,383,psv-eindhoven,2022,2022-11-11,100000,NL1,84,Eredivisie
22912,4742,Christophe,Lepoint,Christophe Lepoint,2715,christophe-lepoint84,Belgium,10/24/84,Central Midfield,right,...,BE1,RFC Seraing,2715,rfc-seraing,2022,2022-11-04,75000,BE1,84,Jupiler Pro League
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386766,1053240,Moussa,Soumano,Moussa Soumano,1147,moussa-soumano05,France,7/9/05,Centre-Forward,NaN,...,FR1,AC Ajaccio,1147,ac-ajaccio,2022,2023-03-27,900000,FR1,05,Ligue 1
92442,1056907,Mauro,Couto,Mauro Couto,2995,mauro-couto05,Portugal,11/15/05,Left Winger,right,...,PO1,FC Pa√ßos de Ferreira,2995,fc-pacos-de-ferreira,2022,2022-11-01,100000,PO1,05,Liga Nos
293169,1063263,Bilal,Bafdili,Bilal Bafdili,354,bilal-bafdili04,NaN,8/3/04,Attacking Midfield,NaN,...,BE1,KV Mechelen,354,kv-mechelen,2022,2023-03-22,100000,BE1,04,Jupiler Pro League
369444,1103981,Nathaniel,Opoku,Nathaniel Opoku,2727,nathaniel-opoku01,Ghana,7/22/01,Centre-Forward,NaN,...,BE1,Oud-Heverlee Leuven,2727,oud-heverlee-leuven,2022,2023-03-22,250000,BE1,01,Jupiler Pro League


In [6]:
new['club_code'] = new['club_code'].replace({'borussia-monchengladbach':"M'Gladbach",'real-betis-sevilla':'Betis',
                                               'heart-of-midlothian-fc':'Hearts','zska-moskau':'CSKA Moscow',
                                              'ogc-nizza':'Nice','inter-mailand':'Inter','hertha-bsc':'Hertha BSC',
                                              'olympique-lyon':'Lyon','ac-florenz':'Fiorentina','ajax-amsterdam':'Ajax',
                                              'fc-paris-saint-germain':'Paris S-G','ural-ekaterinburg':'Ural',
                                             'zenit-st-petersburg':'Zenit','wolverhampton-wanderers':'Wolves',
                                             'west-ham-united':'West Ham','as-rom':'Roma','rasenballsport-leipzig':'RB Leipzig',
                                             'sco-angers':'Angers','vitoria-guimaraes-sc':'Vitória',
                                             'royale-union-saint-gilloise':'Union SG','deportivo-alaves':'no_team'
                                            ,'oud-heverlee-leuven':'OH Leuven','espanyol-barcelona':'Espanyol','ssc-neapel':'Napoli'
                                            ,'stade-brest-29':'Brest','dijon-fco':'no_team','beerschot-v-a-':'no_team'
                                            ,'1-fc-koln':'Köln','fc-watford':'no_team','fc-pacos-de-ferreira':'Paços'
                                            ,'rotor-volgograd':'no_team','fc-crotone':'no_team','krylya-sovetov-samara':'Samara'
                                            ,'fc-metz':'no_Team','rubin-kazan':'no_team','pec-zwolle':'no_team'
                                            ,'cagliari-calcio':'no_team','fk-ufa':'no_team','pfk-tambov':'no_team'
                                            ,'sc-farense':'no_team','huddersfield-town':'no_team','sd-eibar':'no_team'})

In [7]:
from fuzzywuzzy import fuzz

def team_lookup(name, name_list):
    max_score = 0
    name_to_ret = ''
    for team in name_list:
        temp_score = fuzz.ratio(name, team)
        if temp_score > max_score:
            max_score = temp_score
            name_to_ret = team
        
    return name_to_ret

In [8]:
team_list = ['Liverpool','R.F.C. Seraing',"M'Gladbach",'Torpedo Moscow','Betis','Villarreal','Manchester Utd',
     'Famalicão','Rangers','Hibernian','Bayern Munich','Hearts','Monza','Aston Villa','Porto','Marítimo',
     'CSKA Moscow','Sevilla','Nice','Antwerp','Inter','Hertha BSC','Tottenham','Cádiz','Real Madrid','Ajaccio','Lyon',
     'Barcelona','Sassuolo','Salernitana','Milan','Spezia','Sampdoria','Auxerre','Rennes','Casa Pia',
     'Anderlecht','Fiorentina','Ajax','Paris S-G','Ural','Aberdeen','Gil Vicente FC','Strasbourg','Lens',
     'Ross County','Zenit','Chelsea','Wolves','Cremonese','West Ham','Twente','Rayo Vallecano','RKC Waalwijk',
     "Sparta R'dam",'Zulte Waregem','Bologna','Southampton','Lorient','Lille','Westerlo','Everton','Athletic Club',
     'NEC Nijmegen','Augsburg','Fortuna Sittard','Atlético Madrid','Dundee United','Dortmund','Real Sociedad',
     'Wolfsburg','Schalke 04','Kilmarnock','Dynamo Mosc','Marseille','Gent','Lecce','Rio Ave','Sint-Truiden',
     'Eint Frankfurt','Juventus','Celtic','Bochum','St Mirren','Crystal Palace','Leicester City','Oostende',
     'Empoli','Freiburg','Brighton','Hellas Verona','Sporting CP','Lazio','Roma',"Nott'ham Forest",'St Johnstone',
     'Sochi','Nantes','Osasuna','Hoffenheim','Loko Moscow','Udinese','Montpellier','Valencia','Club Brugge',
     'Troyes','Mallorca','Valladolid','Livingston','PSV Eindhoven','Fulham','Portimonense','RB Leipzig','Celta Vigo',
     'Chaves','Benfica','Elche','Nizhny Novgorod','Utrecht','Charleroi','Orenburg','Werder Bremen','Feyenoord',
     'Girona','Almería','Spartak Moscow','Clermont Foot','Leverkusen','Cambuur','Motherwell','Bournemouth',
     'Emmen','Angers','Vitória','Mechelen','Braga','Leeds United','Arouca','Vizela','Atalanta','Union SG',
     'Brentford','Newcastle Utd','Union Berlin','Getafe','Vitesse','Stuttgart','OH Leuven','Espanyol','Monaco',
     'Napoli','Mainz 05','Go Ahead Eag','Manchester City','Fakel Voronezh','Torino','Heerenveen','Arsenal',
     'AZ Alkmaar','Köln','Cercle Brugge','Kortrijk','Akhmat Grozny','Standard Liège','Boavista','Brest','Santa Clara',
     'Toulouse','Estoril','Paços','Eupen','FC Khimki','Groningen','Samara','Volendam','Excelsior','Krasnodar',
     'Rostov','Reims','Genk','no_team']

names = list(new['club_code'])
new_names = []
for name in names:
    new_name = team_lookup(name, team_list)
    new_names.append(new_name)
    
new['Squad'] = new_names

In [10]:
players = pd.read_csv('player_data_2023.csv')
pl = list(players['player_code'].unique())
pn = list(new['player_code'])

new_pl = pl
new_pn = pn.copy()
for i in range(len(pn) - 1, -1, -1):
    name = pn[i]
    np = team_lookup(name, pl)
    if name == np:
        del pn[i]
        pl = list(filter(lambda x: x!=name, pl))
        
replacements = {}
for name in pn:
    np2 = team_lookup(name, pl)
    squad_value = new[new['player_code'] == name]['Squad'].values[0]
    check = players[players['player_code'] == np2]['Squad'].values[0]

    if name[-2:] == np2[-2:] and squad_value == check:
        replacements[name] = np2
    else:
        replacements[name] = 'na'
        
replacements['denis-shepilov00'] = 'na'

new['player_code'] = new['player_code'].replace(replacements)
new

,player_id,first_name,last_name,name,current_club_id,player_code,country_of_birth,date_of_birth,sub_position,foot,...,current_club_name,club_id,club_code,last_season,date,market_value_in_eur,player_club_domestic_competition_id,Born,Based,Squad
359754,3333,James,Milner,James Milner,31,james-milner86,England,1/4/86,Central Midfield,right,...,Liverpool FC,31,fc-liverpool,2022,2022-11-03,2000000,GB1,86,Premier League,Liverpool
149285,3455,Zlatan,Ibrahimoviƒá,Zlatan Ibrahimoviƒá,5,zlatan-ibrahimovic81,Sweden,10/3/81,Centre-Forward,right,...,AC Milan,5,ac-mailand,2022,2022-11-11,2000000,IT1,81,Serie A,Milan
5413,4311,Maarten,Stekelenburg,Maarten Stekelenburg,610,na,Netherlands,9/22/82,Goalkeeper,right,...,Ajax Amsterdam,610,Ajax,2022,2022-11-11,100000,NL1,82,Eredivisie,Ajax
414713,4391,Boy,Waterman,Boy Waterman,383,na,Netherlands,1/24/84,Goalkeeper,right,...,PSV Eindhoven,383,psv-eindhoven,2022,2022-11-11,100000,NL1,84,Eredivisie,PSV Eindhoven
22912,4742,Christophe,Lepoint,Christophe Lepoint,2715,christophe-lepoint84,Belgium,10/24/84,Central Midfield,right,...,RFC Seraing,2715,rfc-seraing,2022,2022-11-04,75000,BE1,84,Jupiler Pro League,R.F.C. Seraing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386766,1053240,Moussa,Soumano,Moussa Soumano,1147,moussa-soumano05,France,7/9/05,Centre-Forward,NaN,...,AC Ajaccio,1147,ac-ajaccio,2022,2023-03-27,900000,FR1,05,Ligue 1,Ajaccio
92442,1056907,Mauro,Couto,Mauro Couto,2995,mauro-meireles-couto05,Portugal,11/15/05,Left Winger,right,...,FC Pa√ßos de Ferreira,2995,Paços,2022,2022-11-01,100000,PO1,05,Liga Nos,Paços
293169,1063263,Bilal,Bafdili,Bilal Bafdili,354,bilal-bafdili04,NaN,8/3/04,Attacking Midfield,NaN,...,KV Mechelen,354,kv-mechelen,2022,2023-03-22,100000,BE1,04,Jupiler Pro League,Mechelen
369444,1103981,Nathaniel,Opoku,Nathaniel Opoku,2727,nathan-opoku01,Ghana,7/22/01,Centre-Forward,NaN,...,Oud-Heverlee Leuven,2727,OH Leuven,2022,2023-03-22,250000,BE1,01,Jupiler Pro League,OH Leuven


In [11]:
new[new['player_code'].str.contains('svinov')]

,player_id,first_name,last_name,name,current_club_id,player_code,country_of_birth,date_of_birth,sub_position,foot,...,current_club_name,club_id,club_code,last_season,date,market_value_in_eur,player_club_domestic_competition_id,Born,Based,Squad
338458,599150,Ilya,Svinov,Ilya Svinov,1124,ilja-svinov00,Russia,9/25/00,Goalkeeper,right,...,Fakel Voronezh,1124,fakel-voronezh,2022,2022-11-21,600000,RU1,00,Premier Liga,Fakel Voronezh


In [12]:
new.to_csv('transfermarkt_data.csv')